In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-218819
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-218819


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "clusterOne"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.01, 0.1, 1),
        '--max_iter': choice(10, 20, 50, 100, 250)
    }
)
# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.', 
                      command=['python', 'train.py'],
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=4,
                                     max_concurrent_runs=4)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ac1caff4-9baa-4e8b-a6fa-368481fe5800
Web View: https://ml.azure.com/runs/HD_ac1caff4-9baa-4e8b-a6fa-368481fe5800?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-218819/workspaces/quick-starts-ws-218819&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-12-16T19:54:10.972680][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-12-16T19:54:11.6591368Z][SCHEDULER][INFO]Scheduling job, id='HD_ac1caff4-9baa-4e8b-a6fa-368481fe5800_0' 
[2022-12-16T19:54:11.7886737Z][SCHEDULER][INFO]Scheduling job, id='HD_ac1caff4-9baa-4e8b-a6fa-368481fe5800_1' 
[2022-12-16T19:54:11.9384621Z][SCHEDULER][INFO]Scheduling job, id='HD_ac1caff4-9baa-4e8b-a6fa-368481fe5800_2' 
[2022-12-16T19:54:12.0387895Z][SCHEDULER][INFO]Scheduling job, id='HD_ac1caff4-9baa-4e8b-a6fa-368481fe5800_3' 
[2022-12-16T19:54:12.1452122Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_ac1caff4-9baa-4e8b-a6fa-368481fe5800_2

{'runId': 'HD_ac1caff4-9baa-4e8b-a6fa-368481fe5800',
 'target': 'clusterOne',
 'status': 'Completed',
 'startTimeUtc': '2022-12-16T19:54:10.284739Z',
 'endTimeUtc': '2022-12-16T19:55:42.738089Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0bb50256-da25-4768-a3fc-8e914e43f965',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': '15',
  'score': '0.91350531107739',
  'best_child_run_id': 'HD_ac1caff4-9baa-4e8b-a6fa-368481fe5800_3',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_ac1caff4-9baa-4e8b-a6fa-368481fe5800_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType

In [6]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

In [7]:
best_run_metrics

{'Regularization Strength:': 1.0,
 'Max iterations:': 100,
 'Accuracy': 0.91350531107739}

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [10]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [14]:
from azureml.train.automl import AutoMLConfig
import pandas as pd

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=pd.concat([x_train, y_train], axis=1),
    label_column_name='y',
    n_cross_validations=4)

In [15]:
# Submit your automl run

auto_run = exp.submit(config=automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_fe15f057-743a-4b91-91c5-0989e02d1881,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


2022/12/16 20:17:23 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2947                          |1                               |26360                                 |
+------------------------------+--------------------------------+-----

2022-12-16:20:41:56,191 INFO     [explanation_client.py:334] Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
********************************************************************************************


In [16]:
# Retrieve and save your best automl model.

best_auto_run, fitted_auto_model = auto_run.get_output()
best_auto_run.get_metrics()

{'precision_score_micro': 0.9179817905918057,
 'balanced_accuracy': 0.781009479093295,
 'norm_macro_recall': 0.56201895818659,
 'AUC_micro': 0.9809388736785629,
 'weighted_accuracy': 0.9519466274545204,
 'accuracy': 0.9179817905918057,
 'f1_score_micro': 0.9179817905918057,
 'log_loss': 0.26145259717826985,
 'average_precision_score_macro': 0.8275473026143173,
 'AUC_weighted': 0.9481718889409558,
 'f1_score_weighted': 0.9168998445076436,
 'recall_score_weighted': 0.9179817905918057,
 'matthews_correlation': 0.576865255321803,
 'f1_score_macro': 0.7881169605182003,
 'recall_score_macro': 0.781009479093295,
 'precision_score_macro': 0.7961985002811971,
 'AUC_macro': 0.9481718889409558,
 'precision_score_weighted': 0.9160721510870078,
 'recall_score_micro': 0.9179817905918057,
 'average_precision_score_weighted': 0.9561197226944138,
 'average_precision_score_micro': 0.9815938532697442,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_fe15f057-743a-4b91-91c5-0989e02d1881_28